In [2]:
import cv2
import time
import numpy as np
import random
import matplotlib.pyplot as plt
from utils.extract_features import get_features_from_filename
from utils.plot_stuff import plot_images, plot_hha_components, plot_angle_directions, plot_masks, showSeSeBBoxes, plot_selected_regions
%matplotlib inline

MIN_SIMILARITY_WITH_GT_REGION = 0.5
MIN_SIMILARITY_WITH_NEIGHBOR_BBOX = 0.5
NeedFastSeSe = False

In [16]:
X__, Y__ = get_features_from_filename("3", print_plots=False, print_info=True)

[INFO] unique label classes: [ 0  2  3  5 15 21 34 35 36 37]
[INFO] number of unique label classes: 10
[INFO] all images loaded in 2.0767s
[INFO] using *quality* selective search
[INFO] total number of region proposals: 6594
[INFO] basic region proposal took 10.3028 seconds
[INFO] label and score calculation took 6.6721s
[INFO] number of boxes after supression: 925
[INFO] non-maximum supression took 2.4808s
[INFO] classes and their counts:
[[  0 906]
 [  2   4]
 [  5   1]
 [ 15   5]
 [ 21   3]
 [ 34   1]
 [ 35   1]
 [ 36   1]
 [ 37   3]]
[INFO] number of classes excluding background :  8
[INFO] feature extraction took 0.4521 seconds
[INFO] X-> (925, 40) 	 Y-> (925,)
[INFO] total time taken 21.9854 seconds


In [17]:
X__, Y__ = get_features_from_filename("69", print_plots=False, print_info=True)

[INFO] unique label classes: [  0  11  21  42  55  59  64  66  74  78  81 119 141 157 158]
[INFO] number of unique label classes: 15
[INFO] all images loaded in 1.9625s
[INFO] using *quality* selective search
[INFO] total number of region proposals: 7492
[INFO] basic region proposal took 8.3346 seconds
[INFO] label and score calculation took 6.1502s
[INFO] number of boxes after supression: 1042
[INFO] non-maximum supression took 3.0868s
[INFO] classes and their counts:
[[   0 1016]
 [  11    1]
 [  21    3]
 [  42    1]
 [  55    2]
 [  59    2]
 [  64    4]
 [  66    2]
 [  74    1]
 [  78    1]
 [  81    3]
 [ 119    1]
 [ 141    2]
 [ 157    1]
 [ 158    2]]
[INFO] number of classes excluding background :  14
[INFO] feature extraction took 0.5105 seconds
[INFO] X-> (1042, 40) 	 Y-> (1042,)
[INFO] total time taken 20.0448 seconds


In [18]:
X__, Y__ = get_features_from_filename("72", print_plots=False, print_info=True)

[INFO] unique label classes: [  0   2   4  21  64  74  85 119 144 157 158]
[INFO] number of unique label classes: 11
[INFO] all images loaded in 1.9989s
[INFO] using *quality* selective search
[INFO] total number of region proposals: 3184
[INFO] basic region proposal took 8.9336 seconds
[INFO] label and score calculation took 2.2131s
[INFO] number of boxes after supression: 529
[INFO] non-maximum supression took 0.7549s
[INFO] classes and their counts:
[[  0 508]
 [  2   1]
 [  4   1]
 [ 21   2]
 [ 64   6]
 [ 74   2]
 [ 85   1]
 [119   3]
 [144   2]
 [157   2]
 [158   1]]
[INFO] number of classes excluding background :  10
[INFO] feature extraction took 0.2870 seconds
[INFO] X-> (529, 40) 	 Y-> (529,)
[INFO] total time taken 14.1878 seconds
